# Explore GPS Data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from sklearn.cluster import DBSCAN
from scipy.stats import mode

import matplotlib.pyplot as plt
from geopy.distance import great_circle

today = date.today()
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")
week_ago = week_ago.strftime("%Y-%m-%d")
today = "06102023"

datapath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"
filepath = datapath + f"export_{today}.csv"

datapath1 = datapath + f"export_tiki_{today}/"
filepath_1 = datapath1 + "epoch_part0001.csv"
filepath_2 = datapath1 + "epoch_part0002.csv"
filepath_3 = datapath1 + "epoch_part0003.csv"
filepath_4 = datapath1 + "epoch_part0004.csv"


Bad key text.latex.preview in file /Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       

In [2]:
df_1 = pd.read_csv(filepath_1, encoding= "latin-1")
df_2 = pd.read_csv(filepath_2, encoding= "latin-1")
df_3 = pd.read_csv(filepath_3, encoding= "latin-1")
df_4 = pd.read_csv(filepath_4, encoding= "latin-1")

/Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_location_1 = df_1[df_1.type.isin(["Latitude", "Longitude"])]
df_location_2 = df_2[df_2.type.isin(["Latitude", "Longitude"])]
df_location_3 = df_3[df_3.type.isin(["Latitude", "Longitude"])]
df_location_4 = df_4[df_4.type.isin(["Latitude", "Longitude"])]

In [4]:
df_active = pd.read_csv("/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/active_participants.csv")
df_active.rename(columns = {"Pseudonym ": "customer", "EMA ID": "ema_id", "Ende EMA Baseline": "end_ema", "Status": "status",
                            "Start EMA Baseline": "start_ema", "Studienversion":"study_version", "FOR_ID":"for_id"}, inplace=True)
df_active["customer"] = df_active["customer"].str[:4]
df_active = df_active[["customer", "end_ema", "start_ema", "status"]]


In [5]:
df_complete = pd.concat([df_location_1, df_location_2, df_location_3,df_location_4])

In [6]:
df_complete.customer.nunique()

91

In [7]:
df_complete = df_complete[["customer", "startTimestamp", "type", "doubleValue", 
                           "timezoneOffset"]]

In [8]:
df_complete["customer"] = df_complete.customer.str.split("@").str.get(0)
df_complete["customer"] = df_complete["customer"].str[:4]

In [9]:
df_complete["timezoneOffset"] = df_complete["timezoneOffset"] * 60000

In [10]:
df_complete["date"] = df_complete["startTimestamp"] + df_complete["timezoneOffset"]

In [11]:
df_complete = pd.merge(df_complete, df_active, on="customer")

In [16]:
df_complete = df_complete.loc[df_complete.status == "Abgeschlossen"]

## Analyze GPS Data

In [19]:
df_int = df_complete.pivot(
    index=["customer", "date"],
    columns="type",
    values="doubleValue")

In [20]:
df_int = df_int.rename_axis(None, axis=1).reset_index()

In [21]:
#df_int= df_int.sort_values(by=["customer", "date"]).drop_duplicates(subset=["date"], keep="last")

In [22]:
df_int["date"] = (pd.to_datetime(df_int["date"],unit='ms'))

In [23]:
df_int.dropna(subset = ['Latitude', 'Longitude', 'date'], inplace=True)

In [24]:
df_int["day"] = df_int.date.dt.strftime('%Y/%m/%d')
df_int["hours"] = df_int.date.dt.hour

In [28]:
df_int.groupby("customer")["day"].nunique()

customer
3ZqH    18
3s90    21
42aH    40
5Ger    17
5qL5    37
Emob    15
Fcz9    16
GuiZ    16
L8UG    21
bNPw    15
bVec    18
f7RF    19
i0to    15
tOQ8    22
uN1K    14
vPEr    16
Name: day, dtype: int64

## Perform DBScan on data from single ID 1

In [ ]:
df_single = df_int.loc[df_int.customer == "0ePW"]

In [ ]:
df_single.shape

In [ ]:
coords = df_single[['Latitude', 'Longitude']].to_numpy()

In [ ]:
# equitorial radius of the earth = 6,378.1 
kms_per_radian = 6371.0088
epsilon = 0.2 / kms_per_radian
min_samples = 10

In [ ]:
db = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_


In [ ]:
cluster_l = pd.Series(cluster_labels)
df_single["cluster_label"] = cluster_l

In [ ]:
num_clusters = len(set(cluster_labels))- (1 if -1 in cluster_labels else 0)
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
# plot the reduced set of coordinate points
fig, ax = plt.subplots()
fig.set_size_inches(10, 6)
rs_scatter = ax.scatter(df_single['Longitude'], df_single['Latitude'], c='m', edgecolor='k', alpha=.4, s=150)
 
# set axis labels, tick labels, and title

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
 
plt.show()

#At this scale, only a few dozen of the 1,759 data points are really visible. 
#Even zoomed in very close, several locations have hundreds of data points stacked directly 
#on top of each other due to the duration of time spent at one location. Unless we are interested in time 
#dynamics, we simply do not need all of these spatially redundant points – they just bloat the data set’s size.

In [ ]:
# plot the reduced set of coordinate points
fig, ax = plt.subplots()
fig.set_size_inches(10, 6)

rs_scatter = ax.scatter(coords[:, 0], coords[:,1], c=cluster_labels,cmap="Paired",alpha=.8, s=100)

 
# set axis labels, tick labels, and title

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
 
plt.show()

## Perform DBScan on data from single ID 2

In [ ]:
df_single2 = df_int.loc[df_int.customer == "gfo4"]

In [ ]:
df_single2.shape

In [ ]:
coords2 = df_single2[['Latitude', 'Longitude']].to_numpy()

In [ ]:
# plot the reduced set of coordinate points
fig, ax = plt.subplots()
fig.set_size_inches(10, 6)
rs_scatter = ax.scatter(df_single2['Longitude'], df_single2['Latitude'], c="m", edgecolor='k', alpha=.4, s=150)
 
# set axis labels, tick labels, and title

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
 
plt.show()

In [ ]:
db2 = DBSCAN(eps=epsilon, min_samples=min_samples, algorithm='ball_tree', metric='haversine').fit(np.radians(coords2))
cluster_labels2 = db2.labels_

In [ ]:
num_clusters2 = len(set(cluster_labels2))- (1 if -1 in cluster_labels2 else 0)

In [ ]:
num_clusters2 = len(set(cluster_labels2))
clusters = pd.Series([coords2[cluster_labels2 == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters2))

In [ ]:
# plot the reduced set of coordinate points
fig, ax = plt.subplots()
fig.set_size_inches(10, 6)
rs_scatter = ax.scatter(df_single2['Longitude'], df_single2['Latitude'], c=cluster_labels2, cmap="plasma", alpha=.2, s=100)
 
# set axis labels, tick labels, and title

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
 
plt.show()

## Apply DBScan to whole dataframe

In [ ]:
df_int["n_hours"] = df_int.groupby(["customer", "day"])["hours"].transform("nunique")

In [ ]:
df_int = df_int.loc[df_int["n_hours"] >= 8]

In [ ]:
df_int["n_days_8"] = df_int.groupby("customer")["day"].transform("nunique")

In [ ]:
df_int = df_int.loc[df_int["n_days_8"] >= 7]

In [ ]:
def assign_day_index(user_group):
    user_group['day_index'] = (user_group["day"] != user_group["day"].shift()).cumsum()
    return user_group

In [ ]:
df_int = df_int.groupby('customer').apply(assign_day_index)

In [ ]:
df_int['hourID'] = df_int['customer'].astype(str) + '0' + \
                    df_int['day_index'].astype(str) + '0' + \
                    df_int['hours'].astype(str)

In [ ]:
df_int.shape

## Transform lon lat coordinates 

In [ ]:
#For spatial analyses focused on Berlin, Germany, you might opt to use the ETRS89 / UTM zone 33N, which is commonly used for Northern Germany. Its EPSG code is 25833.
# Here's a short description: ETRS89: European Terrestrial Reference System 1989. 
#ETRS89 is a geodetic reference system intended for use over the whole of Europe and is an update to 
#the older European Datum 1950. UTM zone 33N: Universal Transverse Mercator coordinate system, 
# zone 33N represents a longitudinal zone of 6 degrees that covers Berlin and is suitable for small to 
# medium-scale mapping efforts, especially when accuracy in metric measurements is required.

# Assuming you have a DataFrame named `data` with 'longitude' and 'latitude'
# Creating a GeoDataFrame, assuming 'lon' and 'lat' are your coordinate column names
geometry = [Point(xy) for xy in zip(df_int['Longitude'], df_int['Latitude'])]
geodata = gpd.GeoDataFrame(df_int, geometry=geometry)

# Defining the CRS for WGS 84 and ETRS89 / UTM zone 33N
wgs84 = {'init': 'epsg:4326'}
utm_33n = {'init': 'epsg:25833'}

# Assigning the initial CRS and reprojecting to UTM 33N
geodata.crs = wgs84
geodata = geodata.to_crs(utm_33n)

In [ ]:
# equitorial radius of the earth = 6,378.1 
kms_per_radian = 6371.0088
epsilon = 0.2 / kms_per_radian
min_samples = 10

geodata['x'] = geodata['geometry'].x
geodata['y'] = geodata['geometry'].y

In [ ]:
# Function for clustering
def db2(x):
    clustering_model = DBSCAN(eps=epsilon, min_samples=min_samples)
    cluster_labels = clustering_model.fit_predict(x[['x', 'y']])
    return pd.DataFrame({'cluster_100m': cluster_labels})

# Group by 'userID' and apply clustering function
geodata_cluster_df = geodata.groupby('customer').apply(lambda x: db2(x)).reset_index()

In [ ]:
geodata_cluster_df

In [ ]:
# Merge this with the main data frame
geodata_clusters = pd.concat([geodata.reset_index(drop=True), geodata_cluster_df['cluster_100m']], axis=1)
geodata_clusters

In [ ]:
# Delete outliers (where cluster_20m == -1)
geodata_clusters = geodata_clusters[geodata_clusters['cluster_100m'] != -1]

In [ ]:
geodata_clusters.groupby("customer").cluster_100m.nunique().describe()

In [ ]:
geodata_clusters.cluster_100m.nunique()

In [ ]:
# Generate unique IDs for clusters
geodata_clusters['clusterID'] = geodata_clusters['customer'].astype(str) + '00' + \
geodata_clusters['cluster_100m'].astype(str)

## Generate Home Location from data

In [ ]:
# Filter data for night hours (midnight to 6:00 am)
geodata_night = geodata_clusters.loc[(geodata_clusters['hours'] >= 0) & (geodata_clusters['hours'] < 6)]

# Find the mode of clusterID per user during night hours
geodata_night = geodata_night.copy()
geodata_night['home'] = geodata_night.groupby('customer')['clusterID'].transform(lambda x: mode(x)[0][0])

In [ ]:
# Calculating various metrics to validate the home cluster
geodata_night['nights_with_obs'] = geodata_night.groupby('customer')['day'].transform('nunique')
geodata_night['night_obs'] = geodata_night.groupby('customer')['day'].transform('size')


In [ ]:
# Finding the frequency of the mode
geodata_night['n_home'] = geodata_night.groupby('customer')['home'].transform(lambda x: x.value_counts().iloc[0])
geodata_night['f_home'] = geodata_night['n_home'] / geodata_night['night_obs']

# Updating the 'home' label based on conditions
geodata_night['home'] = geodata_night.apply(
    lambda x: x['home'] if x['nights_with_obs'] >= 7 and x['f_home'] > 0.5 else None, axis=1
)

# Extracting a mapping of userID to home cluster
user_home_mapping = geodata_night[['customer', 'home']].drop_duplicates()

# Merging back to the full dataset
geodata_clusters = pd.merge(geodata_clusters, user_home_mapping, on='customer', how='left')

In [ ]:
# Grab relevant columns
cbusdata = geodata[['customer', 'date']]
clusterdata = geodata_clusters[['customer', 'date', 'clusterID']]

# Merge dataframes
df_int = pd.merge(df_int, clusterdata, how='left', on=['customer', 'date'])
df_int = pd.merge(df_int, cbusdata, how='left', on=['customer', 'date'])

# Order the data
df_int = df_int.sort_values(by=['customer', 'date'])


In [ ]:
df_int.head(50)

In [ ]:
# Function to shift vectors
def shift_vector(vec, shift):
    if shift > 0:
        return np.concatenate((np.full(shift, np.nan), vec[:-shift]))
    elif shift < 0:
        return np.concatenate((vec[-shift:], np.full(-shift, np.nan)))
    else:
        return vec

# Shift vectors for lat and long
df_int['lat_p1'] = df_int.groupby('customer')['x'].transform(lambda x: shift_vector(x, -1))
df_int['lon_p1'] = df_int.groupby('customer')['y'].transform(lambda x: shift_vector(x, -1))

# haversine formula for distance calculation between two lat,lon points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # radius of Earth in meters
    phi_1 = np.radians(lat1)
    phi_2 = np.radians(lat2)
    
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi/2.0)**2 + np.cos(phi_1) * np.cos(phi_2) * np.sin(delta_lambda/2.0)**2
    
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    meters = R * c  # output distance in meters
    return meters

# Calculate distance (in meters) using the haversine function
df_int['dist_to_prev'] = haversine(df_int['lat'], df_int['lon'], df_int['lat_p1'], df_int['lon_p1'])

# Shift vectors for clusterID
df_int['clusterID_p1'] = df_int.groupby('customer')['clusterID'].transform(lambda x: shift_vector(x, -1))

# Set distance traveled to 0 if the cluster was the same
df_int.loc[df_int['clusterID'] == df_int['clusterID_p1'], 'dist_to_prev'] = 0

# Aggregate distance for each user
agg = df_int.groupby('customer')['dist_to_prev'].sum().reset_index(name='distanceM')
agg['distanceKM'] = agg['distanceM'] / 1000

# Add to main dataframe
df_int = pd.merge(df_int, agg, on='customer')